In [24]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import sys
import io

In [25]:
url = "https://finance.naver.com/sise/"
page = urlopen(url)

soup = BeautifulSoup(page, "html.parser")

top_table = soup.find("table", id="siselist_tab_0")
top_table

<table cellpadding="0" cellspacing="0" class="type_2" id="siselist_tab_0" summary="탑종목 상한가 리스트">
<caption>상한가</caption>
<col width="4%"/>
<col width="4%"/>
<col width="4%"/>
<col width="*"/>
<col width="7%"/>
<col width="9%"/>
<col width="7%"/>
<col width="12%"/>
<col width="12%"/>
<col width="12%"/>
<col width="12%"/>
<tr>
<th>순위</th>
<th>연속</th>
<th>누적</th>
<th>종목명</th>
<th>현재가</th>
<th class="tr">전일비  </th>
<th>등락률</th>
<th>거래량</th>
<th>시가</th>
<th>고가</th>
<th>저가</th>
</tr>
<tr><td class="blank_08" colspan="10"></td></tr>
<tr>
<td class="tc"><img alt="01" height="11" src="https://ssl.pstatic.net/imgstock/images5/ico_n01.gif" width="11"/></td>
<td class="number">2</td>
<td class="number">2</td>
<td><a class="tltle" href="/item/main.nhn?code=036180" onclick="clickcr(this, 'bqu.2', '036180', '1', event)">지더블유바이텍</a></td>
<td class="number">4,065</td>
<td class="number">
<em class="bu_p bu_pup2" style="margin:0 4px 0 0"><span class="blind">상한</span></em><span class="tah p11 red01">
				

In [26]:
# 컬럼 인덱스 추출

columns_info = top_table.find("tr")
columns_names = columns_info.find_all("th")
column_index = []
for column in columns_names:
    column_index.append(column.text)
print(column_index)

['순위', '연속', '누적', '종목명', '현재가', '전일비\xa0\xa0', '등락률', '거래량', '시가', '고가', '저가']


In [27]:
# 상위 주식 종목 정보 추출

tr_list = top_table.find_all("tr")
stock_info = []
for i in range(0, 12):
    one_stock_info = []
    td_list = tr_list[i].find_all("td")
    for td in td_list:
        one_stock_info.append(td.text)
    stock_info.append(one_stock_info)

stock_info.pop(0)
stock_info.pop(0)
stock_info.pop(-3)
stock_info.pop(-3)
stock_info.pop(-3)

print(stock_info)

[['', '2', '2', '지더블유바이텍', '4,065', '\n상한\n\t\t\t\t935\n\t\t\t\t\n', '\n\n\t\t\t\t+29.87%\n\t\t\t\t\n', '503,160', '4,065', '4,065', '4,065'], ['', '1', '1', '정원엔시스', '3,540', '\n상한\n\t\t\t\t815\n\t\t\t\t\n', '\n\n\t\t\t\t+29.91%\n\t\t\t\t\n', '30,021,206', '2,730', '3,540', '2,710'], ['', '1', '2', 'SGA', '1,510', '\n상한\n\t\t\t\t345\n\t\t\t\t\n', '\n\n\t\t\t\t+29.61%\n\t\t\t\t\n', '50,561,716', '1,150', '1,510', '1,130'], ['', '1', '2', '에스엔피제네틱스', '5,870', '\n상한\n\t\t\t\t760\n\t\t\t\t\n', '\n\n\t\t\t\t+14.87%\n\t\t\t\t\n', '1', '5,870', '5,870', '5,870'], ['', '1', '2', '크로넥스', '5,810', '\n상한\n\t\t\t\t750\n\t\t\t\t\n', '\n\n\t\t\t\t+14.82%\n\t\t\t\t\n', '32,170', '5,190', '5,810', '5,190'], ['', '1', '17', '썬테크', '9,750', '\n상한\n\t\t\t\t1,270\n\t\t\t\t\n', '\n\n\t\t\t\t+14.98%\n\t\t\t\t\n', '4', '9,750', '9,750', '7,410'], ['', '1', '3', '삼성스팩2호', '7,690', '\n상한\n\t\t\t\t1,770\n\t\t\t\t\n', '\n\n\t\t\t\t+29.90%\n\t\t\t\t\n', '23,265,316', '6,230', '7,690', '5,840']]


In [28]:
# 추출한 데이터의 빈문자열, 공백문자열 제거 후 딕셔너리 형태로 데이터 담기

stock_final = {}
n = 1
for stock in stock_info:
    
    stock.pop(0)
    stock[4] = stock[4].replace("\n", "")
    stock[4] = stock[4].replace("\t", "")
    
    stock[5] = stock[5].replace("\n", "")
    stock[5] = stock[5].replace("\t", "")
    
    stock_final[n] = stock
    n += 1
    
print(stock_final)

{1: ['2', '2', '지더블유바이텍', '4,065', '상한935', '+29.87%', '503,160', '4,065', '4,065', '4,065'], 2: ['1', '1', '정원엔시스', '3,540', '상한815', '+29.91%', '30,021,206', '2,730', '3,540', '2,710'], 3: ['1', '2', 'SGA', '1,510', '상한345', '+29.61%', '50,561,716', '1,150', '1,510', '1,130'], 4: ['1', '2', '에스엔피제네틱스', '5,870', '상한760', '+14.87%', '1', '5,870', '5,870', '5,870'], 5: ['1', '2', '크로넥스', '5,810', '상한750', '+14.82%', '32,170', '5,190', '5,810', '5,190'], 6: ['1', '17', '썬테크', '9,750', '상한1,270', '+14.98%', '4', '9,750', '9,750', '7,410'], 7: ['1', '3', '삼성스팩2호', '7,690', '상한1,770', '+29.90%', '23,265,316', '6,230', '7,690', '5,840']}


In [29]:
# 데이터 프레임 형태로 변환 및 컬럼 인덱스 설정, 엑셀 및 csv 파일로 저장

top_stock_df = pd.DataFrame(stock_final).transpose()
top_stock_df = top_stock_df.reset_index(drop=False)
top_stock_df = top_stock_df.set_axis(column_index, axis=1, inplace=False)
top_stock_df.to_csv("Top 종목 상한가 정보.csv", index=False)
top_stock_df.to_excel("Top 종목 상한가 정보.xlsx", index=False)
top_stock_df

,순위,연속,누적,종목명,현재가,전일비,등락률,거래량,시가,고가,저가
0,1,2,2,지더블유바이텍,"4,065",상한935,+29.87%,"503,160","4,065","4,065","4,065"
1,2,1,1,정원엔시스,"3,540",상한815,+29.91%,"30,021,206","2,730","3,540","2,710"
2,3,1,2,SGA,"1,510",상한345,+29.61%,"50,561,716","1,150","1,510","1,130"
3,4,1,2,에스엔피제네틱스,"5,870",상한760,+14.87%,1,"5,870","5,870","5,870"
4,5,1,2,크로넥스,"5,810",상한750,+14.82%,"32,170","5,190","5,810","5,190"
5,6,1,17,썬테크,"9,750","상한1,270",+14.98%,4,"9,750","9,750","7,410"
6,7,1,3,삼성스팩2호,"7,690","상한1,770",+29.90%,"23,265,316","6,230","7,690","5,840"
